In [1]:
#Creating DataFrame and varibles for ep->epippim(P) 

import ROOT, sys

ROOT.gStyle.SetGridColor(17)
ROOT.gStyle.SetPadGridX(1)
ROOT.gStyle.SetPadGridY(1)

ROOT.gStyle.SetPadRightMargin(0.01)
ROOT.gStyle.SetPadLeftMargin(0.075)
ROOT.gStyle.SetPadBottomMargin(0.12)

ROOT.gStyle.SetOptFit(1)

ROOT.TH1.AddDirectory(0)


fname =  "/work/clas12/kenjo/trains/root/eFPimFPip/lvl1_epimpip.nSidis_005*"

dname = "outb" if 'outb' in fname else 'inb'

rdf = ROOT.RDataFrame("h22",fname)

vals = "pe,the,fie,ppip,thpip,fipip,ppim,thpim,fipim,angPimPip,phi_t"
vals += ",angPimPip_LF,angPimPip_CM"
vals += ",Q2,xb,ww,mt"
vals += ",mmpro,misse"
vals += ",mrho,rhoPhi_CM"

rdf = rdf.Define("vals","""
double E0 = 10.6041, Mpro = 0.938272, Mele = 0.000511, Mpi = 0.13957, Mpi0 = 0.1349769;
TLorentzVector beam, targ;
beam.SetXYZM(0,0,E0,Mele);
targ.SetXYZM(0,0,0,Mpro);


TLorentzVector ele, pip, pim;
ele.SetXYZM(ex, ey, ez, Mele);
//pro.SetXYZM(prox, proy, proz, Mpro);
pip.SetXYZM(pipx, pipy, pipz, Mpi);
pim.SetXYZM(pimx, pimy, pimz, Mpi);

double pe = ele.P(), the=ele.Theta()*TMath::RadToDeg(), fie=ele.Phi()*TMath::RadToDeg();
double ppip = pip.P(), thpip=pip.Theta()*TMath::RadToDeg(), fipip=pip.Phi()*TMath::RadToDeg();
double ppim = pim.P(), thpim=pim.Theta()*TMath::RadToDeg(), fipim=pim.Phi()*TMath::RadToDeg();




double mrho = (pip+pim).M();
//double mdelta = (pro+pip).M();

//double mm2pip = (beam+targ-ele-pro-pim).M2();
//double mm2pim = (beam+targ-ele-pro-pip).M2();
double mmpro = (beam+targ-ele-pip-pim).M();
double misse = (beam+targ-ele-pip-pim).E();
//double mepx = (beam+targ-ele-pro).M();


auto q = (beam-ele);
double Q2 = - (beam-ele).M2();
double ww = (beam+targ-ele).M();
double xb = Q2/(ww*ww - targ.M2() + Q2);
double mt = - ((beam+targ-ele-pip-pim)-targ).M2();


double Theta_q = q.Theta();
double Phi_el = ele.Phi();

auto Rot_Matrix = [&](TLorentzVector vector, int Lab2CM_or_CM2Lab, double Theta_Rot, double Phi_Rot){ 

        double Rot_X1 = vector.X();
        double Rot_Y1 = vector.Y();
        double Rot_Z1 = vector.Z();

        double Rot_X = Rot_X1;
        double Rot_Y = Rot_Y1;
        double Rot_Z = Rot_Z1;



        // Lab2CM_or_CM2Lab is a parameter which determines if you rotating from the lab frame to the CM frame, or if you are rotating back in the opposite direction
        // Lab2CM_or_CM2Lab = -1 gives a rotation to the CM frame (from the lab frame)
        // Lab2CM_or_CM2Lab = +1 gives a rotation to the lab frame (from the CM frame)


        Theta_Rot = -1*Theta_Rot;   // Always give the angle of rotation Theta as the value given by .Theta()
                                    // This subroutine will handle the fact that the matrix rotation wants the negative of the angle of rotation


        // Rotation to Lab Frame
        if(Lab2CM_or_CM2Lab == -1){
            Rot_X = Rot_X1*TMath::Cos(Theta_Rot)*TMath::Cos(Phi_Rot) - Rot_Z1*TMath::Sin(Theta_Rot) + Rot_Y1*TMath::Cos(Theta_Rot)*TMath::Sin(Phi_Rot);
            Rot_Y = Rot_Y1*TMath::Cos(Phi_Rot) - Rot_X1*TMath::Sin(Phi_Rot);
            Rot_Z = Rot_Z1*TMath::Cos(Theta_Rot) + Rot_X1*TMath::Cos(Phi_Rot)*TMath::Sin(Theta_Rot) + Rot_Y1*TMath::Sin(Theta_Rot)*TMath::Sin(Phi_Rot);
        }


        // Rotation to CM Frame
        if(Lab2CM_or_CM2Lab == 1){
            Rot_X = Rot_X1*TMath::Cos(Theta_Rot)*TMath::Cos(Phi_Rot) + Rot_Z1*TMath::Cos(Phi_Rot)*TMath::Sin(Theta_Rot) - Rot_Y1*TMath::Sin(Phi_Rot);
            Rot_Y = Rot_Y1*TMath::Cos(Phi_Rot) + Rot_X1*TMath::Sin(Phi_Rot)*TMath::Cos(Theta_Rot) + Rot_Z1*TMath::Sin(Theta_Rot)*TMath::Sin(Phi_Rot);
            Rot_Z = Rot_Z1*TMath::Cos(Theta_Rot) - Rot_X1*TMath::Sin(Theta_Rot);
        }



        TLorentzVector vector_Rotated(Rot_X, Rot_Y, Rot_Z, vector.E());

        return vector_Rotated;


    };


auto angPimPip_LF = pim.Vect().Angle(pip.Vect())*TMath::RadToDeg();

auto rho = pip+pim;


// Center of Mass Frame
auto beam_Clone = Rot_Matrix(beam, -1, Theta_q, Phi_el);
auto targ_Clone = Rot_Matrix(targ, -1, Theta_q, Phi_el);
auto ele_Clone  = Rot_Matrix(ele,  -1, Theta_q, Phi_el);
auto pip_Clone = Rot_Matrix(pip,  -1, Theta_q, Phi_el);
auto pim_Clone = Rot_Matrix(pim,  -1, Theta_q, Phi_el);
auto q_Clone = Rot_Matrix(q,  -1, Theta_q, Phi_el);
auto rho_Clone = Rot_Matrix(rho,  -1, Theta_q, Phi_el);






auto fCM = q_Clone + targ_Clone;
auto boost = -(fCM.BoostVector());


auto qlv_Boost(q_Clone);
auto ele_Boost(ele_Clone);
auto beamBoost(beam_Clone);
auto targBoost(targ_Clone);
auto pipBoost(pip_Clone);
auto pimBoost(pim_Clone);
auto rhoBoost(rho_Clone);




qlv_Boost.Boost(boost);
ele_Boost.Boost(boost);
beamBoost.Boost(boost);
targBoost.Boost(boost);
pipBoost.Boost(boost);
pimBoost.Boost(boost);
rhoBoost.Boost(boost);



auto rhoPhi_CM = rhoBoost.Vect().Phi()*TMath::RadToDeg();



auto angPimPip_CM = pimBoost.Vect().Angle(pipBoost.Vect())*TMath::RadToDeg();




TVector3 v0, v1;
v0 = qlv_Boost.Vect().Cross(ele_Boost.Vect());
v1 = qlv_Boost.Vect().Cross(pipBoost.Vect()+pimBoost.Vect());
Double_t c0, c1, c2, c3;
c0 = v0.Dot(pipBoost.Vect()+pimBoost.Vect());
c1 = v0.Dot(v1);
c2 = v0.Mag();
c3 = v1.Mag();


double phi_t = (c0/TMath::Abs(c0)) * TMath::ACos(c1 /(c2*c3))*TMath::RadToDeg();
if(phi_t <= 0){
    phi_t = phi_t +360;
}








// Rho Frame
auto Phi_rho =  rhoBoost.Phi();
auto Theta_rho = rhoBoost.Theta();

TLorentzVector rhoBoost_r;
rhoBoost_r.SetXYZM(rhoBoost.Vect().x(), rhoBoost.Vect().y(), rhoBoost.Vect().z(), 0.7754);

auto pip_Clone2 = Rot_Matrix(pipBoost,  -1, Theta_rho, Phi_rho);
auto pim_Clone2 = Rot_Matrix(pimBoost,  -1, Theta_rho, Phi_rho);
auto rho_Clone2 = Rot_Matrix(rhoBoost_r,  -1, Theta_rho, Phi_rho);


auto fCM2 = -rho_Clone2;
auto boost2 = -(fCM2.BoostVector());

auto pipBoost2(pip_Clone2);
auto pimBoost2(pim_Clone2);

pipBoost2.Boost(boost2);
pimBoost2.Boost(boost2);
auto angPimPip = pimBoost2.Vect().Angle(pipBoost2.Vect())*TMath::RadToDeg();





return vector<double> {"""+vals+"};")



phcuts = "angEPim>10 && angEPip>10"

for iv, vname in enumerate(vals.split(',')):
    rdf = rdf.Define(vname, "vals[{}]".format(iv))
    
rdf = rdf.Filter("Q2>0.8 && ww>1.8")
    




Welcome to JupyROOT 6.22/06


In [2]:
import numpy as np


# Modify root code:https://root.cern/doc/master/combinedFit_8py_source.html
#2 arrays which follow the pattern of how many fit paramters 

#Here there are 13 parameters 7 of which are shared (p1,p2,p4,p5,p6,p7,p8)
iparP = np.array([0,1,2,3,4,5,6,7,8,9], dtype=np.int32) 
iparM = np.array([10,1,2,11,4,5,6,7,8,12],dtype=np.int32)



#Custum class that makes a chi2 for both fits

class GlobalChi2(object):
    def __init__(self, f1, f2):
        self._f1 = f1
        self._f2 = f2
 
    def __call__(self, par):
        # parameter vector is first background (in common 1 and 2) and then is
        # signal (only in 2)
 
        # the zero-copy way to get a numpy array from a double *
        par_arr = np.frombuffer(par, dtype=np.float64, count=len(iparP)+len(iparM))
 
        p1 = par_arr[iparP]
        p2 = par_arr[iparM]
 
        return self._f1(p1) + self._f2(p2)
        
    

In [3]:

def fit1d(h1,h2,num,hel = True):
   # 3 pieces to the over all fit
    fy1 = ROOT.TF1("fy1","breitwigner(0)",0.28,2.5)
    fy1.SetParameters(2000,0.77549,0.15)
    fy1.SetLineColor(1)
    fy1.SetLineStyle(2)
    
    
    fy2 = ROOT.TF1("fy2","breitwigner(0)",0.28,2.5)
    fy2.SetParameters(1,1.275,1.867)
    fy2.FixParameter(1,1.275)
    fy2.SetLineColor(3)
    fy2.SetLineStyle(3)
    
    
    fy3 = ROOT.TF1("fy3","(x-2*0.1396)^[3]*exp(pol2(0))",0.28,2.5)
    fy3.SetLineColor(4)
    fy3.SetLineStyle(4)
    
    # Current best overall fit, 2 one for Positve helicity and one for Negative

    fyp = ROOT.TF1("fy","breitwigner(0)+breitwigner(3)+(x-2*0.1396)^[9]*exp(pol2(6))",0.28,2.5)
    fym = ROOT.TF1("fy","breitwigner(0)+breitwigner(3)+(x-2*0.1396)^[9]*exp(pol2(6))",0.28,2.5)
    fyp.SetParameters(100,0.77549,0.15,1,1.275,20,6,3,-5,1)
    fym.SetParameters(100,0.77549,0.15,1,1.275,20,6,3,-5,1)
    
    #Setting Parameters names
    fyp.SetParName(0,"#rhoAmp")
    fyp.SetParName(1,"#rho Mean")
    fyp.SetParName(2,"#rho Sigma")

    fyp.SetParName(3,"f2 Amp")
    fyp.SetParName(4,"f2 Mean")
    fyp.SetParName(5,"f2 Sigma")

    fyp.SetParName(6,"bg 0")
    fyp.SetParName(7,"bg 1")
    fyp.SetParName(8,"bg 2")
    fyp.SetParName(9,"bg n")
    
    
    fym.SetParName(0,"#rhoAmp")
    fym.SetParName(1,"#rho Mean")
    fym.SetParName(2,"#rho Sigma")

    fym.SetParName(3,"f2 Amp")
    fym.SetParName(4,"f2 Mean")
    fym.SetParName(5,"f2 Sigma")

    fym.SetParName(6,"bg 0")
    fym.SetParName(7,"bg 1")
    fym.SetParName(8,"bg 2")
    fym.SetParName(9,"bg n")
    
    
    
    #taking DataFrame's 1DHisto and making it into TH1D
    hp = ROOT.TH1D()
    hp = h1.GetPtr()
    
    hm = ROOT.TH1D()
    hm = h2.GetPtr()
    
    
    #Wrapper for Fits
    wfyp = ROOT.Math.WrappedMultiTF1(fyp, 1)
    wfym = ROOT.Math.WrappedMultiTF1(fym, 1)
    
    
    #Fitting Data for fits
    opt = ROOT.Fit.DataOptions();
    rang = ROOT.Fit.DataRange();
    rang.SetRange(0.28,2.5)
    
    dataP = ROOT.Fit.BinData(opt,rang)
    ROOT.Fit.FillData(dataP,hp)


    dataM = ROOT.Fit.BinData(opt,rang)
    ROOT.Fit.FillData(dataM,hm)
    
    #fitting data using Chi2 functions
    chiP = ROOT.Fit.Chi2Function(dataP,wfyp)
    chiM = ROOT.Fit.Chi2Function(dataM,wfym)
    
    
    #Making global Chi2 in custum class
    globalChi2 = GlobalChi2(chiP, chiM)
    
    fitter = ROOT.Fit.Fitter()
    
    
    #Init Fit Parameters for overall chi2 fit
    Npar = 13
    par0 = np.array([100,0.77549,0.15,1,1.275,20,6,3,-5,1,100,1,1])
    
    
    fitter.Config().SetParamsSettings(Npar, par0)
    
    #Minimizing chi2 fits
    fitter.Config().MinimizerOptions().SetPrintLevel(0)
    fitter.Config().SetMinimizer("Minuit2", "Migrad")
    
    
    #in order to fit global Chi2 function it needs to be in a wrapper
    #Functor fails in pyroot for root version 6.24 or less
    #globalChi2Functor = ROOT.Math.Functor(globalChi2, Npar) Faster way of doing it for v6.26 and up 
    ROOT.gInterpreter.Declare("""
    ROOT::Math::Functor foo(const std::function<double(double const *)> &x, int num) { return ROOT::Math::Functor(x,num); }
    //ROOT::Math::Functor foo(GlobalChi2 x, int num) { return ROOT::Math::Functor(x,num); }
    
    """)
     
    
    #calls ROOT functor in ROOT
    globalChi2Functor = ROOT.foo(globalChi2,Npar)
    
    
    #making fit for our Global Chi2 function
    fitter.FitFCN(globalChi2Functor, 0, dataP.Size() + dataM.Size(), True)
    result = fitter.Result()
    result.Print(ROOT.std.cout)
    
    
    
   
    
    
    """fy1.FixParameter(0,fy.GetParameter(0))
    fy1.FixParameter(1,fy.GetParameter(1))
    fy1.FixParameter(2,fy.GetParameter(2))

    
    fy2.FixParameter(0,fy.GetParameter(3))
    fy2.FixParameter(1,fy.GetParameter(4))
    fy2.FixParameter(2,fy.GetParameter(5))
    
    fy3.FixParameter(0,fy.GetParameter(6))
    fy3.FixParameter(1,fy.GetParameter(7))
    fy3.FixParameter(2,fy.GetParameter(8))
    fy3.FixParameter(3,fy.GetParameter(9))
    
    h1.Fit(fy1,"QR+")
    h1.Fit(fy2,"QR+")
    h1.Fit(fy3,"QR+")

    print("Hi")
    backUp = h1.Clone()
    h1.Add(fy2,-1)
    h1.Add(fy3,-1)"""
    
    #fyPara = fy.GetParameters()
    #fyParaEr = fy.GetParErrors()
    
    
    #Printing our histograms with Fits
    c1 = ROOT.TCanvas("Simfit", "Simultaneous fit of two histograms", 10, 10, 700, 700)
    
    
    c1.Divide(1, 2)
    c1.Draw()
    c1.cd(1)
    fyp.SetFitResult(result, iparP)
    fyp.SetRange(rang().first, rang().second)
    
    fyp.SetLineColor(ROOT.kBlue)
    hp.GetListOfFunctions().Add(fyp)
    hp.Draw()
 
    c1.cd(2)
    
    fym.SetFitResult(result, iparM)
    fym.SetRange(rang().first, rang().second)
    
    
    fym.SetLineColor(ROOT.kBlue)
    hm.GetListOfFunctions().Add(fym)
    hm.Draw()
    
    c1.SaveAs("combinedFit.png")

In [4]:
cut2 = rdf.Filter("mmpro < 1.2 && mmpro > 0.85")
cut2 = cut2.Filter("mt < 1")
temp = cut2.Filter("2.3 < ww < 2.6")
temp = temp.Filter("phi_t > 315 && phi_t < 360.0")
pos = temp.Filter("ihel > 0")
neg = temp.Filter("ihel < 0")



hmrho_p = pos.Histo1D(("hmrho","invariant mass: #pi^{+}+#pi^{-},Heli+ Wbin:",200,0,2.5), "mrho")
print("Pos fit:")
#fy_p,fy_pErr,hmrho_p,backup_p = fit1d(hmrho_p,num+1,fy[1],fy[2],fy[4],fy[5],fy[6],fy[7],fy[8],False)

hmrho_m = neg.Histo1D(("hmrho","invariant mass: #pi^{+}+#pi^{-},Heli- Wbin:",200,0,2.5), "mrho")
print("Neg fit:")
#fy_m,fy_mErr,hmrho_m,backup_m = fit1d(hmrho_m,num+1,fy[1],fy[2],fy[4],fy[5],fy[6],fy[7],fy[8],False)

fit1d(hmrho_p,hmrho_m,1)


Pos fit:
Neg fit:

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      740.768
NDf                       =          329
Edm                       =  7.77246e-06
NCalls                    =         1299
Par_0                     =      158.759   +/-   3.33649     
Par_1                     =     0.763775   +/-   0.000868278 
Par_2                     =     0.133908   +/-   0.00258858  
Par_3                     =      25.7857   +/-   2.27903     
Par_4                     =      1.31135   +/-   0.00547576  
Par_5                     =      0.20873   +/-   0.0136461   
Par_6                     =      6.62542   +/-   0.127093    
Par_7                     =     0.227416   +/-   0.175302    
Par_8                     =     -1.40486   +/-   0.0585904   
Par_9                     =     0.825185   +/-   0.0322686   
Par_10                    =      158.325   +/-   3.27494     
Par_11                    =      28.9534   +/-   2.28451     
Pa